In [1]:
from breeze_connect import BreezeConnect
import numpy as np
import pandas as pd
import pandas_ta as ta
from datetime import date, datetime, timedelta
import time
import csv
import logging
from tenacity import retry, stop_after_attempt, wait_exponential
import json, yaml, time
import os

from breeze_connect import BreezeConnect
breeze = BreezeConnect(api_key="64mV026553514z5565%7S258@^4l3753")
import urllib
breeze.generate_session(api_secret="20286x551)P23443543722J0t28s90D3",
                          session_token="51349293")


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('trading_strategy_new.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


TIME_1 = datetime.strptime("09:15", "%H:%M").time()
TIME_2 = datetime.strptime("15:20", "%H:%M").time()

order = 0  # For CE
order2 = 0  # For PE
sl = 0    # For CE
sl2 = 0   # For PE
QTY =75
spot_price=None
tick_data={}
breeze.ws_connect()





def on_ticks(ticks):
    global tick_data,spot_price
    if 'strike_price' in ticks:
        data = ticks['strike_price']+'_'+ticks['right']
        if data in tick_data:
            tick_data[data]=ticks['last']
    else:
        spot_price=ticks['last']

        # print(ticks)


breeze.on_ticks=on_ticks

def initiate_ws( atm,right=''):
    global tick_data
    if right=='call':    
        # print("hello")
        leg=breeze.subscribe_feeds(exchange_code="NFO",
                                stock_code="NIFTY",
                                product_type="options",
                                # expiry_date=f'{expiry}T06:00:00.000Z',
                                expiry_date=expiry1,
                                right="call",
                                strike_price=str(atm),
                                get_exchange_quotes=True,
                                get_market_depth=False)
        tick_data[str(atm)+'_Call']=''
    # print(leg)
    elif right=='put':
        leg2=breeze.subscribe_feeds(exchange_code="NFO",
                                stock_code="NIFTY",
                                product_type="options",
                                expiry_date=expiry1,
                                right="put",
                                strike_price=str(atm),
                                get_exchange_quotes=True,
                                get_market_depth=False)
        tick_data[str(atm)+'_Put']=''
        # print(leg2)
    elif right=='others':
        leg3=breeze.subscribe_feeds(exchange_code="NSE",
                                stock_code="NIFTY",
                                product_type="CASH",
                                expiry_date="",
                                right="others",
                                get_exchange_quotes=True,
                                get_market_depth=False)
    time.sleep(1)

def deactivate_ws( atm,right=''):
    global tick_data
    if right=='call':    
        breeze.unsubscribe_feeds(exchange_code="NFO",
                            stock_code="NIFTY",
                            product_type="options",
                            expiry_date=expiry1,
                            right="call",
                            strike_price=str(atm),
                            get_exchange_quotes=True,
                            get_market_depth=False)
        tick_data.pop(str(atm)+'_Call')
    elif right=='put':
        breeze.unsubscribe_feeds(exchange_code="NFO",
                            stock_code="NIFTY",
                            product_type="options",
                            expiry_date=expiry1,
                            right="put",
                            strike_price=str(atm),
                            get_exchange_quotes=True,
                            get_market_depth=False)
        tick_data.pop(str(atm)+'_Put')

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def nifty_spot():
    try:
        if spot_price is not None:
            return float(spot_price)
        raise ValueError("No success response from API")
    except Exception as e:
        logging.error(f"Error fetching Nifty spot: {str(e)}")
        raise


@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def option_historical(ce_or_pe, strike, yesterday, today):
    try:
        option_data = breeze.get_historical_data_v2(
            interval="1minute",
            from_date=f"{yesterday}T07:00:00.000Z",
            to_date=f"{today}T17:00:00.000Z",
            stock_code="NIFTY",
            exchange_code="NFO",
            product_type="options",
            expiry_date=f"{EXPIRY}T07:00:00.000Z",
            right=ce_or_pe,
            strike_price=strike
        )
        time.sleep(4)
        if option_data.get('Success'):
            df = pd.DataFrame(option_data['Success'])
            print(f"Historical data fetched for {ce_or_pe} at strike {strike}")
            return df
        raise ValueError("No success response from API")
    except Exception as e:
        logger.error(f"Error fetching historical data for {ce_or_pe} {strike}: {str(e)}")
        print(f"Error fetching historical data for {ce_or_pe} {strike}: {str(e)}")
        raise


def write_to_csv(data):
    csv_file = "rsi_supertrend_RPS.csv"
    headers = ['Date', 'Entry Time', 'Strike', 'CE or PE', 'Entry premium', 
              'Exit Time', 'Exit premium', 'PnL', 'Quantity']
    try:
        try:
            with open(csv_file, 'x', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(headers)
                print("Created new CSV file")
        except FileExistsError:
            pass
        with open(csv_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(data)
            print(f"Trade data written to CSV: {data}")
    except Exception as e:
        logger.error(f"Error writing to CSV: {str(e)}")
        print(f"Error writing to CSV: {str(e)}")


@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def fetch_nearest_expiry(stock_code, exchange_code, product_type, right, strike):
    try:
        response = breeze.get_option_chain_quotes(
            stock_code=stock_code,
            exchange_code=exchange_code,
            product_type=product_type,
            right=right,
            strike_price=strike
        )
        time.sleep(2)
        if not isinstance(response, dict) or response.get("Status") != 200 or not response.get("Success"):
            error_msg = response.get("Error", "Unknown error") if isinstance(response, dict) else "Invalid response format"
            print(f"Error fetching data: {error_msg}")
            return None
        
        try:
            expiry_dates = {item["expiry_date"] for item in response["Success"] if "expiry_date" in item}
        except (KeyError, TypeError) as e:
            print(f"Error processing response data: {e}")
            return None    
        if not expiry_dates:
            print("No expiry dates found in the response.")
            return None
        
        current_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        nearest_expiry = None
        min_days_diff = float('inf')
        for expiry in expiry_dates:
            try:
                expiry_date = datetime.strptime(expiry, "%d-%b-%Y")
                days_diff = (expiry_date - current_date).days
                if 0 <= days_diff < min_days_diff:
                    min_days_diff = days_diff
                    nearest_expiry = expiry
            except (ValueError, TypeError) as e:
                print(f"Error parsing date '{expiry}': {e}")
                continue
        if nearest_expiry:
            return nearest_expiry
        print("No future expiry dates found.")
        return None
    except Exception as e:
        print(f"Unexpected error in fetch_nearest_expiry: {str(e)}")
        return None

def main():
    global order, order2, sl, sl2,tick_data, EXPIRY, expiry1
    initiate_ws('0','others')
    nifty = nifty_spot()
    atm = str(round(nifty / 50) * 50)
    expiry1 = fetch_nearest_expiry("NIFTY", "NFO", "options", "call", atm)
    EXPIRY = datetime.strptime(expiry1, '%d-%b-%Y')
    EXPIRY = EXPIRY.strftime('%Y-%m-%d')
    print("Expiry is:", EXPIRY)
    while True:
        try:
            now = datetime.now()
            current_time = now.time()
            
            if TIME_1 <= current_time < TIME_2:
                today = now.strftime('%Y-%m-%d')
                yesterday = (now - timedelta(days=5)).strftime('%Y-%m-%d')
                
                # CE Entry Logic
                if order == 0 and now.second == 0:
                    try:
                        nifty = nifty_spot()
                        print('Nifty spot:', nifty)
                        atm = round(nifty / 50) * 50
                        ce_otm = atm + 100
                        print(f"CE OTM Strike: {ce_otm}")
                        
                        ce_option = option_historical("call", ce_otm, yesterday, today)
                        ce_option.ta.rsi(close='close', length=14, append=True)
                        supertrend = ta.supertrend(ce_option['high'], ce_option['low'], 
                                                ce_option['close'], length=10, multiplier=2)
                        ce_option['supertrend'] = supertrend['SUPERTd_10_2.0']
                        ce_option['volume_avg'] = ce_option['volume'].rolling(window=5).mean()
                        ce_option['volume_check'] = (ce_option['volume'] > 1.5 * ce_option['volume_avg']).astype(int)
                        
                        last_row = ce_option.iloc[-1]
                        rsi_condition = (last_row['RSI_14'] - ce_option.iloc[-2]['RSI_14']) > 10
                        supertrend_condition = last_row['supertrend'] == 1
                        volume_condition = last_row['volume_check'] == 1
                        print(f"CE Conditions - RSI: {last_row['RSI_14']} (>70: {rsi_condition}), "
                              f"Supertrend: {last_row['supertrend']} (1: {supertrend_condition}), "
                              f"Volume Check: {last_row['volume_check']} (1: {volume_condition})")
                        
                        if rsi_condition and supertrend_condition and volume_condition:
                            entry_time = now.strftime('%H:%M:%S')
                            buy_price = last_row['close']
                            logger.info(f"Call entry at {buy_price} for strike {ce_otm}")
                            print(f"CE Entry executed at {buy_price}")
                            initiate_ws(ce_otm,'call')
                            data_key=str(ce_otm)+'_Call'
                            order = 1
                            sl = 0
                            ce_entry_data = [today, entry_time, ce_otm, f"call_{EXPIRY}", buy_price]
                        else:
                            print(now, f"No CE position taken - Conditions not met")
                    except Exception as e:
                        logger.error(f"Error in CE entry logic: {str(e)}")
                        print(f"Error in CE entry: {str(e)}")
                
                # CE Exit Logic
                if order == 1:
                    try:
                        time.sleep(0.2)
                        if now.second == 0 :
                            ce_option = option_historical("call", ce_otm, yesterday, today)
                            ce_option.ta.rsi(close='close', length=14, append=True)
                            supertrend = ta.supertrend(ce_option['high'], ce_option['low'], 
                                                    ce_option['close'], length=10, multiplier=2)
                            ce_option['supertrend'] = supertrend['SUPERTd_10_2.0']
                            last_row = ce_option.iloc[-1]
                            time.sleep(1)
                        
                        if data_key in tick_data:
                            leg1_cmp=tick_data[data_key]
                            print(f"CE Current Market Price: {leg1_cmp}")
                            
                            profit_condition = (leg1_cmp - buy_price) >= 10
                            if profit_condition:
                                sl = 1
                                print("CE SL triggered (profit >= 10)")
                            sl_hit = sl == 1 and leg1_cmp <= buy_price
                            if sl_hit:
                                sl = 2
                                logger.info("SL Hit for CE")
                                print("CE SL Hit")
                            
                            rsi_exit = last_row['RSI_14'] < 70
                            supertrend_exit = last_row['supertrend'] != 1
                            time_exit = current_time >= datetime.strptime("15:19", "%H:%M").time()
                            print(f"CE Exit Conditions - RSI: {last_row['RSI_14']} (<70: {rsi_exit}), "
                                f"Supertrend: {last_row['supertrend']} (!=1: {supertrend_exit}), "
                                f"Time Exit: {time_exit}, SL: {sl == 2}")
                            
                            if rsi_exit or supertrend_exit or time_exit or sl == 2:
                                sell_price = leg1_cmp
                                exit_time = now.strftime('%H:%M:%S')
                                pnl = round(sell_price - buy_price, 2)
                                logger.info(f"Call exit, PnL: {pnl}")
                                print(f"CE Exit executed at {sell_price}, PnL: {pnl}")
                                write_to_csv(ce_entry_data + [exit_time, sell_price, pnl, QTY])
                                deactivate_ws(ce_otm,'call')
                                order = 0
                            else:
                                print(now, "No CE exit - Conditions not met")
                        else:
                            print(f"Strike:{data_key} not in tick data:{tick_data} ")
                    except Exception as e:
                        logger.error(f"Error in CE exit logic: {str(e)}")
                        print(f"Error in CE exit: {str(e)}")
                
                # PE Entry Logic
                if order2 == 0 and now.second == 0:
                    try:
                        nifty = nifty_spot()
                        atm_strike = round(nifty / 50) * 50
                        pe_otm = atm_strike - 100
                        print(f"PE OTM Strike: {pe_otm}")
                        
                        pe_option = option_historical("put", pe_otm, yesterday, today)
                        pe_option.ta.rsi(close='close', length=14, append=True)
                        supertrend = ta.supertrend(pe_option['high'], pe_option['low'], 
                                                pe_option['close'], length=10, multiplier=2)
                        pe_option['supertrend'] = supertrend['SUPERTd_10_2.0']
                        pe_option['volume_avg'] = pe_option['volume'].rolling(window=5).mean()
                        pe_option['volume_check'] = (pe_option['volume'] > 1.5 * pe_option['volume_avg']).astype(int)
                        
                        last_row_pe = pe_option.iloc[-1]
                        rsi_condition = (last_row_pe['RSI_14'] - pe_option.iloc[-2]['RSI_14']) > 10
                        supertrend_condition = last_row_pe['supertrend'] == 1
                        volume_condition = last_row_pe['volume_check'] == 1
                        print(f"PE Conditions - RSI: {last_row_pe['RSI_14']} (>70: {rsi_condition}), "
                              f"Supertrend: {last_row_pe['supertrend']} (1: {supertrend_condition}), "
                              f"Volume Check: {last_row_pe['volume_check']} (1: {volume_condition})")
                        
                        if rsi_condition and supertrend_condition and volume_condition:
                            entry_time = now.strftime('%H:%M:%S')
                            buy_pe_price = last_row_pe['close']
                            logger.info(f"Put entry at {buy_pe_price} for strike {pe_otm}")
                            print(f"PE Entry executed at {buy_pe_price}")
                            initiate_ws(pe_otm,'put')
                            data_key=str(pe_otm)+'_Put'
                            order2 = 1
                            sl2 = 0
                            pe_entry_data = [today, entry_time, pe_otm, f"put_{EXPIRY}", buy_pe_price]
                        else:
                            print(now, "No PE position taken - Conditions not met")
                    except Exception as e:
                        logger.error(f"Error in PE entry logic: {str(e)}")
                        print(f"Error in PE entry: {str(e)}")
                
                # PE Exit Logic
                if order2 == 1:
                    try:
                        time.sleep(0.2)
                        if now.second == 0 :
                            pe_option = option_historical("put", pe_otm, yesterday, today)
                            pe_option.ta.rsi(close='close', length=14, append=True)
                            supertrend = ta.supertrend(pe_option['high'], pe_option['low'], 
                                                    pe_option['close'], length=10, multiplier=2)
                            pe_option['supertrend'] = supertrend['SUPERTd_10_2.0']
                            last_row_pe = pe_option.iloc[-1]
                            time.sleep(1)
                        
                        if data_key in tick_data:
                            leg2_cmp=tick_data[data_key]
                            print(f"PE Current Market Price: {leg2_cmp}")
                            
                            profit_condition = (leg2_cmp - buy_pe_price) >= 15
                            if profit_condition:
                                sl2 = 1
                                print("PE SL triggered (profit >= 15)")
                            sl_hit = sl2 == 1 and leg2_cmp <= buy_pe_price
                            if sl_hit:
                                sl2 = 2
                                logger.info("SL Hit for PE")
                                print("PE SL Hit")
                            
                            rsi_exit = last_row_pe['RSI_14'] < 70
                            supertrend_exit = last_row_pe['supertrend'] != 1
                            time_exit = current_time >= datetime.strptime("15:19", "%H:%M").time()
                            print(f"PE Exit Conditions - RSI: {last_row_pe['RSI_14']} (<70: {rsi_exit}), "
                                f"Supertrend: {last_row_pe['supertrend']} (!=1: {supertrend_exit}), "
                                f"Time Exit: {time_exit}, SL: {sl2 == 2}")
                            
                            if rsi_exit or supertrend_exit or time_exit or sl2 == 2:
                                sell_pe_price = leg2_cmp
                                exit_time = now.strftime('%H:%M:%S')
                                pnl = round(sell_pe_price - buy_pe_price, 2)
                                order2 = 0
                                logger.info(f"Put exit, PnL: {pnl}")
                                print(f"PE Exit executed at {sell_pe_price}, PnL: {pnl}")
                                write_to_csv(pe_entry_data + [exit_time, sell_pe_price, pnl, QTY])
                                deactivate_ws(pe_otm,'put')
                            else:
                                print(now, "No PE exit - Conditions not met")
                        else:
                            print(f"Strike:{data_key} not in tick data:{tick_data} ")
                    except Exception as e:
                        logger.error(f"Error in PE exit logic: {str(e)}")
                        print(f"Error in PE exit: {str(e)}")
            else:
                print("Outside trading hours")
            
            time.sleep(1)
            
        except Exception as e:
            logger.error(f"Main loop error: {str(e)}")
            print(f"Main loop error: {str(e)}")
            time.sleep(60)

if __name__ == "__main__":
    main()

exchange_code_name 4. token value NIFTY 50
Expiry is: 2025-04-30
Nifty spot: 24358.1
CE OTM Strike: 24450
Historical data fetched for call at strike 24450
CE Conditions - RSI: 47.235315400037074 (>70: False), Supertrend: -1 (1: False), Volume Check: 0 (1: False)
2025-04-30 14:57:00.737277 No CE position taken - Conditions not met
PE OTM Strike: 24250
Historical data fetched for put at strike 24250
PE Conditions - RSI: 31.903614659334 (>70: False), Supertrend: -1 (1: False), Volume Check: 0 (1: False)
2025-04-30 14:57:00.737277 No PE position taken - Conditions not met
Nifty spot: 24373.5
CE OTM Strike: 24450
Historical data fetched for call at strike 24450
CE Conditions - RSI: 52.59177446498961 (>70: False), Supertrend: -1 (1: False), Volume Check: 0 (1: False)
2025-04-30 14:58:00.894971 No CE position taken - Conditions not met
PE OTM Strike: 24300
Historical data fetched for put at strike 24300
PE Conditions - RSI: 31.15963471951684 (>70: False), Supertrend: -1 (1: False), Volume Che

KeyboardInterrupt: 